<a href="https://colab.research.google.com/github/Iqra-01/Google_AI_studio_Genai/blob/main/templates/Street%20Food%20Scavenger%20Hunt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IkdlbmVyYXRlIGEgc2NhdmVuZ2VyIGh1bnQgZm9yIHN0cmVldCBmb29kIGFyb3VuZCB0aGUgY2l0eSBvZiBTZW91bCwgS29yZWEuIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiIyMgU2VvdWwgU3RyZWV0IEZvb2QgU2NhdmVuZ2VyIEh1bnRcXG5cXG4qKldlbGNvbWUgdG8gU2VvdWwhKiogR2V0IHJlYWR5IHRvIGV4cGxvcmUgdGhlIGNpdHkncyB2aWJyYW50IHN0cmVldCBmb29kIHNjZW5lIHdpdGggdGhpcyBleGNpdGluZyBzY2F2ZW5nZXIgaHVudC4gIFlvdSdsbCBiZSBzZWFyY2hpbmcgZm9yIHNvbWUgb2YgU2VvdWwncyBtb3N0IGJlbG92ZWQgdHJlYXRzLCBhbGwgd2hpbGUgZGlzY292ZXJpbmcgaGlkZGVuIGNvcm5lcnMgYW5kIGV4cGVyaWVuY2luZyB0aGUgY2l0eSdzIGF1dGhlbnRpYyBmbGF2b3JzLlxcblxcbioqSW5zdHJ1Y3Rpb25zOioqXFxuXFxuMS4gUHJpbnQgdGhpcyBzaGVldCBhbmQgYnJpbmcgaXQgd2l0aCB5b3UuXFxuMi4gQ29tcGxldGUgZWFjaCBjaGFsbGVuZ2UgYnkgZmluZGluZyB0aGUgc3BlY2lmaWMgc3RyZWV0IGZvb2QgaXRlbSBkZXNjcmliZWQuIFxcbjMuIE9uY2UgeW91IGZpbmQgaXQsIHRha2UgYSBwaG90byBvZiB5b3Vyc2VsZiB3aXRoIHRoZSBmb29kIVxcbjQuIEhhdmUgZnVuIGV4cGxvcmluZyBTZW91bCdzIGRlbGljaW91cyBzaWRlIVxcblxcbioqQ2hhbGxlbmdlczoqKlxcblxcbioqMS4gVGhlIFNwaWN5IFN0YXJ0OioqIEZpbmQgdGhlIGZpZXJ5IGdvb2RuZXNzIG9mICoqVHRlb2tib2traSoqIChzcGljeSByaWNlIGNha2VzKS4gXFxuXFxuKipCb251cyBDaGFsbGVuZ2U6KiogIFRyeSBpdCB3aXRoIGEgc2lkZSBvZiAqKk9kZW5nKiogKGZpc2ggY2FrZSkgZm9yIGV4dHJhIGZsYXZvci5cXG5cXG4qKjIuIFN3ZWV0IGFuZCBTYXZvcnk6KiogRGlzY292ZXIgdGhlIHVuaXF1ZSBjb21iaW5hdGlvbiBvZiBzd2VldG5lc3MgYW5kIHNhdm9yeSBpbiAqKktpbWJhcCoqIChzZWF3ZWVkIHJpY2Ugcm9sbHMpLlxcblxcbioqQm9udXMgQ2hhbGxlbmdlOioqICBGaW5kIG9uZSB3aXRoIGEgc3BlY2lhbCBpbmdyZWRpZW50IGxpa2Uga2ltY2hpIG9yIHNwYW0uXFxuXFxuKiozLiBUaGUgQ3Jpc3B5IERlbGlnaHQ6KiogTG9jYXRlIHRoZSBpcnJlc2lzdGlibGUgY3J1bmNoIG9mICoqSG90dGVvayoqIChzd2VldCBhbmQgc2F2b3J5IGZpbGxlZCBwYW5jYWtlKS5cXG5cXG4qKkJvbnVzIENoYWxsZW5nZToqKiAgRmluZCBhIEhvdHRlb2sgZmlsbGVkIHdpdGggYSBzdXJwcmlzaW5nIGluZ3JlZGllbnQgbGlrZSBwZWFudXQgYnV0dGVyIG9yIGNob2NvbGF0ZS5cXG5cXG4qKjQuICBUaGUgQ29vbCBEb3duOioqIEZpbmQgYSByZWZyZXNoaW5nICoqQmluZ3N1KiogKHNoYXZlZCBpY2UgZGVzc2VydCkuXFxuXFxuKipCb251cyBDaGFsbGVuZ2U6KiogIFRyeSBhIEJpbmdzdSB3aXRoIGEgdW5pcXVlIGZsYXZvciBsaWtlIGdyZWVuIHRlYSBvciBtYW5nby5cXG5cXG4qKjUuIFRoZSBGdXNpb24gVHJlYXQ6KiogIFNlZWsgb3V0IGEgbW9kZXJuIHR3aXN0IG9uIGEgY2xhc3NpYyBLb3JlYW4gc3RyZWV0IGZvb2QuIFRoaXMgY291bGQgYmUgYW55dGhpbmcgZnJvbSBhIGZ1c2lvbiAqKktpbWNoaSBEb2cqKiB0byBhICoqQnVsZ29naSBCdXJyaXRvKiouXFxuXFxuKipCb251cyBDaGFsbGVuZ2U6KiogIEZpbmQgYSBzdHJlZXQgZm9vZCBzdGFuZCB3aXRoIGEgY3JlYXRpdmUgYW5kIHVuaXF1ZSBmdXNpb24gZGlzaC5cXG5cXG4qKjYuIFRoZSBMb2NhbCBGYXZvcml0ZToqKiAgRGlzY292ZXIgYSBoaWRkZW4gZ2VtIC0gYSBzdHJlZXQgZm9vZCB0aGF0J3MgcG9wdWxhciBhbW9uZyBsb2NhbHMgYnV0IGxlc3Mgd2VsbC1rbm93biB0byB0b3VyaXN0cy4gQXNrIGFyb3VuZCBmb3IgcmVjb21tZW5kYXRpb25zISBcXG5cXG4qKkJvbnVzIENoYWxsZW5nZToqKiAgVHJ5IHNvbWV0aGluZyB5b3UndmUgbmV2ZXIgc2VlbiBiZWZvcmUgYW5kIHNoYXJlIHlvdXIgZXhwZXJpZW5jZSB3aXRoIHlvdXIgZnJpZW5kcyFcXG5cXG4qKjcuIFRoZSBVbHRpbWF0ZSBTbmFjazoqKiAgRmluZCB0aGUgdWx0aW1hdGUgY29tZm9ydCBmb29kLCB0aGUgKipCdW5nZW9wcGFuZyoqIChmaXNoLXNoYXBlZCBwYXN0cnkpLlxcblxcbioqQm9udXMgQ2hhbGxlbmdlOioqICBGaW5kIGEgQnVuZ2VvcHBhbmcgZmlsbGVkIHdpdGggYSBzdXJwcmlzaW5nIGZpbGxpbmcgbGlrZSByZWQgYmVhbiBwYXN0ZSBvciBjdXN0YXJkLlxcblxcbioqOC4gVGhlIFN3ZWV0IEVuZGluZzoqKiBGaW5kIGEgZGVsaWNpb3VzIEtvcmVhbiBkZXNzZXJ0IGxpa2UgKipZdWd3YSoqIChob25leS1nbGF6ZWQgcGFzdHJ5KSBvciAqKllha2d3YSoqIChkZWVwLWZyaWVkIGNvb2tpZSkuXFxuXFxuKipCb251cyBDaGFsbGVuZ2U6KiogIFRyeSBhIGRlc3NlcnQgd2l0aCBhIHRyYWRpdGlvbmFsIEtvcmVhbiBmbGF2b3IgbGlrZSBnaW5zZW5nIG9yIGdpbmdlci5cXG5cXG4qKkNvbmdyYXR1bGF0aW9ucyEqKiBZb3UndmUgY29tcGxldGVkIHlvdXIgU2VvdWwgU3RyZWV0IEZvb2QgU2NhdmVuZ2VyIEh1bnQuIFlvdSd2ZSBub3Qgb25seSB0YXN0ZWQgdGhlIGJlc3Qgb2YgU2VvdWwncyBzdHJlZXQgZm9vZCBidXQgYWxzbyBleHBlcmllbmNlZCBpdHMgdmlicmFudCBjdWx0dXJlIGFuZCBoaWRkZW4gZ2Vtcy4gIFxcblxcbioqRG9uJ3QgZm9yZ2V0IHRvIHNoYXJlIHlvdXIgcGhvdG9zIG9uIHNvY2lhbCBtZWRpYSB3aXRoIHRoZSBoYXNodGFnICNTZW91bFN0cmVldEZvb2RIdW50ISoqIn1dfSx7InJvbGUiOiJ1c2VyIiwicGFydHMiOlt7InRleHQiOiJzdHJlZXQgZm9vZCBpbiBkdWJhaSBnbG9hYmFsIHZpbGxhZ2Ugc2NhdmFuZ2VyIGh1bnQifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiIjIyAgR2xvYmFsIFZpbGxhZ2UgU3RyZWV0IEZvb2QgU2NhdmVuZ2VyIEh1bnQ6IEEgVGFzdGUgb2YgdGhlIFdvcmxkXFxuXFxuR2V0IHJlYWR5IHRvIGVtYmFyayBvbiBhIGN1bGluYXJ5IGFkdmVudHVyZSB0aHJvdWdoIEdsb2JhbCBWaWxsYWdlISAgVGhpcyBzY2F2ZW5nZXIgaHVudCB3aWxsIHRha2UgeW91IG9uIGEgZGVsaWNpb3VzIGpvdXJuZXksIHNhbXBsaW5nIHN0cmVldCBmb29kIGZyb20gYXJvdW5kIHRoZSB3b3JsZC4gXFxuXFxuKipJbnN0cnVjdGlvbnM6KipcXG5cXG4xLiBQcmludCB0aGlzIHNoZWV0IGFuZCBicmluZyBpdCB3aXRoIHlvdSB0byBHbG9iYWwgVmlsbGFnZS5cXG4yLiBDb21wbGV0ZSBlYWNoIGNoYWxsZW5nZSBieSBmaW5kaW5nIHRoZSBzcGVjaWZpYyBzdHJlZXQgZm9vZCBpdGVtIGRlc2NyaWJlZC4gXFxuMy4gT25jZSB5b3UgZmluZCBpdCwgdGFrZSBhIHBob3RvIG9mIHlvdXJzZWxmIHdpdGggdGhlIGZvb2QhXFxuNC4gRW5qb3kgdGhlIGRpdmVyc2UgZmxhdm9ycyBhbmQgY3VsdHVyZXMgYXQgR2xvYmFsIFZpbGxhZ2UhXFxuXFxuKipDaGFsbGVuZ2VzOioqXFxuXFxuKioxLiAgVGhlIE1pZGRsZSBFYXN0ZXJuIERlbGlnaHQ6KiogIFNlZWsgb3V0IGEgKipTaGF3YXJtYSoqIChzcGljZWQgbWVhdCB3cmFwcGVkIGluIHBpdGEgYnJlYWQpIHdpdGggYWxsIHRoZSBmaXhpbmdzLlxcblxcbioqQm9udXMgQ2hhbGxlbmdlOioqIEZpbmQgYSBTaGF3YXJtYSB3aXRoIGEgdW5pcXVlIGZpbGxpbmcgbGlrZSBsYW1iIG9yIGNoaWNrZW4uXFxuXFxuKioyLiAgVGhlIFNwaWN5IFNlbnNhdGlvbjoqKiBEaXNjb3ZlciB0aGUgZmllcnkgZmxhdm9ycyBvZiAqKkluZGlhbiBTdHJlZXQgRm9vZCoqIGxpa2UgKipTYW1vc2FzKiogKGNyaXNweSBmcmllZCBwYXN0cmllcyBmaWxsZWQgd2l0aCBwb3RhdG9lcyBhbmQgcGVhcykgb3IgKipQYW5pIFB1cmkqKiAoaG9sbG93IGNyaXNweSBzaGVsbHMgZmlsbGVkIHdpdGggc3BpY2VkIHdhdGVyIGFuZCBwb3RhdG9lcykuIFxcblxcbioqQm9udXMgQ2hhbGxlbmdlOioqICBUcnkgYSBkaWZmZXJlbnQgdHlwZSBvZiBJbmRpYW4gc3RyZWV0IGZvb2QgbGlrZSAqKkRhaGkgUHVyaSoqIG9yICoqUGF2IEJoYWppKiouXFxuXFxuKiozLiAgVGhlIFN3ZWV0IFRyZWF0OioqIEZpbmQgYSBkZWxpY2lvdXMgKipUdXJraXNoIERlbGlnaHQqKiAoZ2VsYXRpbm91cyBjYW5keSBmbGF2b3JlZCB3aXRoIHJvc2Ugd2F0ZXIgb3IgY2l0cnVzKS5cXG5cXG4qKkJvbnVzIENoYWxsZW5nZToqKiBGaW5kIGEgdW5pcXVlIGZsYXZvciBvZiBUdXJraXNoIERlbGlnaHQgbGlrZSBwaXN0YWNoaW8gb3IgaGF6ZWxudXQuIFxcblxcbioqNC4gIFRoZSAgSW50ZXJuYXRpb25hbCBGdXNpb246KiogIEZpbmQgYSBmb29kIGl0ZW0gdGhhdCBjb21iaW5lcyBmbGF2b3JzIGZyb20gdHdvIGRpZmZlcmVudCBjdWx0dXJlcywgbGlrZSBhICoqU3VzaGkgQnVycml0byoqIG9yIGEgKipLb3JlYW4gRnJpZWQgQ2hpY2tlbiBUYWNvKiouXFxuXFxuKipCb251cyBDaGFsbGVuZ2U6KiogIFRyeSBhIGZ1c2lvbiBkaXNoIGZyb20gYSBjb3VudHJ5IHlvdSd2ZSBuZXZlciBleHBsb3JlZCBiZWZvcmUuXFxuXFxuKio1LiAgVGhlICBTd2VldCBTdXJwcmlzZToqKiBEaXNjb3ZlciBhIGRlc3NlcnQgZnJvbSBhIGNvdW50cnkgeW91IG1pZ2h0IG5vdCBleHBlY3QsIGxpa2UgYSAqKkZyZW5jaCBNYWNhcm9uKiogb3IgYSAqKkphcGFuZXNlIE1vY2hpKiouXFxuXFxuKipCb251cyBDaGFsbGVuZ2U6KiogIEZpbmQgYSBkZXNzZXJ0IHdpdGggYSB1bmlxdWUgZmxhdm9yIG9yIGluZ3JlZGllbnQuXFxuXFxuKio2LiAgVGhlIENsYXNzaWMgQ29tZm9ydCBGb29kOioqICBGaW5kIGEgYmVsb3ZlZCBzdHJlZXQgZm9vZCBmcm9tIGFueSBjb3VudHJ5LCBsaWtlIGEgKipIb3QgRG9nKiogb3IgYSAqKlBpenphIFNsaWNlKiouXFxuXFxuKipCb251cyBDaGFsbGVuZ2U6KiogIEZpbmQgYSB1bmlxdWUgdHdpc3Qgb24gYSBjbGFzc2ljIGNvbWZvcnQgZm9vZC5cXG5cXG4qKjcuICBUaGUgTG9jYWwgVHJlYXN1cmU6KiogIFNlZWsgb3V0IGEgc3RyZWV0IGZvb2QgZnJvbSB0aGUgVW5pdGVkIEFyYWIgRW1pcmF0ZXMsIGxpa2UgYSAqKkx1cWFpbWF0KiogKHN3ZWV0IGR1bXBsaW5ncykgb3IgYSAqKlJlZ2FnKiogKHRoaW4gZmxhdGJyZWFkKS5cXG5cXG4qKkJvbnVzIENoYWxsZW5nZToqKiAgVHJ5IGEgRW1pcmF0aSBkaXNoIHlvdSd2ZSBuZXZlciBoZWFyZCBvZiBiZWZvcmUgYW5kIGFzayB0aGUgdmVuZG9yIGFib3V0IGl0cyBoaXN0b3J5LlxcblxcbioqOC4gIFRoZSBVbHRpbWF0ZSBCaXRlOioqICBGaW5kIGEgZm9vZCBpdGVtIHRoYXQgaXMgY29tcGxldGVseSBuZXcgdG8geW91IGFuZCBleHBhbmQgeW91ciBjdWxpbmFyeSBob3Jpem9ucyFcXG5cXG4qKkJvbnVzIENoYWxsZW5nZToqKiAgVHJ5IHNvbWV0aGluZyB5b3UndmUgbmV2ZXIgZXZlbiBoZWFyZCBvZiBiZWZvcmUgYW5kIHNoYXJlIHlvdXIgZXhwZXJpZW5jZSB3aXRoIGZyaWVuZHMhXFxuXFxuKipDb25ncmF0dWxhdGlvbnMhKiogWW91J3ZlIGNvbXBsZXRlZCB5b3VyIEdsb2JhbCBWaWxsYWdlIFN0cmVldCBGb29kIFNjYXZlbmdlciBIdW50ISBZb3UndmUgdGFzdGVkIHRoZSB3b3JsZCBhbmQgZXhwZXJpZW5jZWQgdGhlIGRpdmVyc2UgYW5kIGRlbGljaW91cyBjdWx0dXJlcyByZXByZXNlbnRlZCBhdCBHbG9iYWwgVmlsbGFnZS5cXG5cXG4qKkRvbid0IGZvcmdldCB0byBzaGFyZSB5b3VyIHBob3RvcyBvbiBzb2NpYWwgbWVkaWEgd2l0aCB0aGUgaGFzaHRhZyAjR2xvYmFsVmlsbGFnZUZvb2RIdW50ISoqXFxuXFxuIn1dfV0=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "Generate a scavenger hunt for street food around the city of Seoul, Korea."
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
                "text": "## Seoul Street Food Scavenger Hunt\\n\\n**Welcome to Seoul!** Get ready to explore the city's vibrant street food scene with this exciting scavenger hunt.  You'll be searching for some of Seoul's most beloved treats, all while discovering hidden corners and experiencing the city's authentic flavors.\\n\\n**Instructions:**\\n\\n1. Print this sheet and bring it with you.\\n2. Complete each challenge by finding the specific street food item described. \\n3. Once you find it, take a photo of yourself with the food!\\n4. Have fun exploring Seoul's delicious side!\\n\\n**Challenges:**\\n\\n**1. The Spicy Start:** Find the fiery goodness of **Tteokbokki** (spicy rice cakes). \\n\\n**Bonus Challenge:**  Try it w

## Call `generate_content`

In [5]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

if response.candidates:
    display(Markdown(response.text))
else:
    # Print prompt feedback if available to understand the blocking reason
    if response.prompt_feedback:
        print(f"Prompt feedback: {response.prompt_feedback}")
    else:
        print("Error: No candidates returned. The prompt may have been blocked.")

Error: No candidates returned. The prompt may have been blocked.


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [6]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

Generate a scavenger hunt for street food around the city of Seoul, Korea. 

-------------------------------------------------------------------------------- 

Role: model 

## Seoul Street Food Scavenger Hunt\n\n**Welcome to Seoul!** Get ready to explore the city's vibrant street food scene with this exciting scavenger hunt.  You'll be searching for some of Seoul's most beloved treats, all while discovering hidden corners and experiencing the city's authentic flavors.\n\n**Instructions:**\n\n1. Print this sheet and bring it with you.\n2. Complete each challenge by finding the specific street food item described. \n3. Once you find it, take a photo of yourself with the food!\n4. Have fun exploring Seoul's delicious side!\n\n**Challenges:**\n\n**1. The Spicy Start:** Find the fiery goodness of **Tteokbokki** (spicy rice cakes). \n\n**Bonus Challenge:**  Try it with a side of **Odeng** (fish cake) for extra flavor.\n\n**2. Sweet and Savory:** Discover the unique combination 